Mount Drive

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import datetime
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import pandas
import sklearn.metrics as metrics
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from numpy import asarray, hstack, vstack
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold
import pandas as pd
import keras 
import tensorflow
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# step_feat = ['goals_conceded_shift','npg_shift','form','bonus_shift','bps_shift','clean_sheets_shift','assists_shift','total_points_shift_to_value','npg_shift_to_value','influence_shift_to_value','value','bps_shift_to_value','bonus_shift_to_value','influence_shift','yellow_cards_shift','goals_scored_shift','goals_scored_shift_to_value','minutes_shift','penalties_saved_shift','saves_shift','own_goals_shift','red_cards_shift','position_DEF','transfers_out','xGChain_shift','Player_Rank_No','total_points_shift_per_minute','bps_shift_per_minute','team_Spurs','team_Wolves','team_Chelsea','bonus_shift_per_minute','opponent_team_Brighton','xA_shift','Supporters','team_Newcastle','team_Man Utd','team_Everton','team_Brighton','opponent_team_West Ham','threat_shift','transfers_balance','position_GK']
# get_step_data = FunctionTransformer(lambda x: x[step_feat], validate=False)

def plot_results(true_value, predicted_value, title):
    plt.figure(figsize=(10, 10))
    plt.scatter(true_value, predicted_value, c='crimson')
    p1 = max(max(predicted_value), max(true_value))
    p2 = min(min(predicted_value), min(true_value))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('True Values', fontsize=15)
    plt.ylabel('Predictions', fontsize=15)
    plt.title(title)
    plt.axis('equal')
    plt.show()

def regression_results(y_true, y_pred, model_string):
    print('# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>')
    print(model_string)
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance, 4))
    print('r2: ', round(r2, 4))
    print('MAE: ', round(mean_absolute_error, 4))
    print('MSE: ', round(mse, 4))
    print('RMSE: ', round(np.sqrt(mse), 4))

def baseline_model():
    model = Sequential()
    model.add(Dense(500, activation= "relu",))
    model.add(Dense(500, activation= "relu"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def get_models():
    models = list()
    # models.append(('SW', Pipeline([('selector', get_step_data), ('LR', LinearRegression())])))   
    # models.append(SVR(C=100, epsilon=0.01, kernel='rbf'))
    # models.append(KNeighborsRegressor(n_neighbors=3))
    # models.append(KerasRegressor(build_fn=baseline_model, epochs=50, verbose=False,validation_split=0.2))
    models.append(LinearRegression())   
    models.append(LinearRegression())   

    return models

def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(), list()
    # define split of data
    kfold = KFold(n_splits=5, shuffle=True)
    # enumerate splits
    for train_ix, test_ix in kfold.split(X):
        fold_yhats = list()
        # get data
        train_X, test_X = X.iloc[train_ix, :], X.iloc[test_ix, :]
        train_y, test_y = y.iloc[train_ix], y.iloc[test_ix]
        meta_y.extend(test_y)
        # fit and make predictions with each sub-model
        for model in models:
            print('Training', model.__class__.__name__) # TODO: GET FEAT
            model.fit(train_X, train_y)  #
            yhat = model.predict(test_X)  
            fold_yhats.append(yhat.reshape(len(yhat), 1)) # store columns
        meta_X.append(hstack(fold_yhats))
    return vstack(meta_X), asarray(meta_y)

def fit_base_models(X, y, models):
    for model in models:
        model.fit(X, y)
    return models

def fit_meta_model(X, y): # Meta learner
    model = LinearRegression()
    model.fit(X, y)
    return model

def evaluate_models(X, y, models, std_y):
    for model in models:
        yhat = np.round(std_y.inverse_transform(model.predict(X)))
        y = np.round(std_y.inverse_transform(y))
        mse = mean_squared_error(y, yhat)
        print('%s: RMSE %.3f' % (model.__class__.__name__, sqrt(mse)))

def super_learner_predictions(X, models, meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict(X)
        meta_X.append(yhat.reshape(len(yhat),1))
    meta_X = hstack(meta_X)
    # predict
    return meta_model.predict(meta_X)

def scale_datasets(df):
  scaled_cols = []
  for col in df.columns.drop('total_points'):
    if df[col].nunique() > 2:
      scaled_cols.append(col)
  std_pred, std_resp = StandardScaler(), StandardScaler()
  X_train, y_train = df.drop('total_points', axis=1), df['total_points'] 
  X_train[scaled_cols] = pd.DataFrame(std_pred.fit_transform(X_train[scaled_cols]), 
                                      columns=df[scaled_cols].columns,
                                      index=X_train.index)
  y_train = pd.DataFrame(std_resp.fit_transform(y_train.to_numpy().reshape(-1, 1)),
                                                columns=['total_points'], index=y_train.index)
  return X_train, y_train, std_resp

def dont_scale_datasets(df):
  X_train, y_train = df.drop('total_points', axis=1), df['total_points'] 
  return X_train, y_train

def preprocess_data(df):
  # Drop indexing features
  idxrs = df[['player_name', 'kickoff_time', 'season', 'GW']]
  df = df.drop(columns = ['player_name', 'kickoff_time', 'season', 'GW'])
  # Binary encoding
  for col in df.columns:
    df[col] = df[col].replace({True:1, False:0})
  # One hot encodings
  ohe_cols = []
  for col in df.select_dtypes(include='object').columns:
      ohe_cols.append(col)
  df = pd.get_dummies(df, columns=ohe_cols, prefix=ohe_cols)
  df['kickoff_time'] = idxrs['kickoff_time']
  df['GW'] = idxrs['GW']
  return df


def get_training_data(GW, scale):
    df = pd.read_csv('./drive/MyDrive/Data/engineered_us.csv')
    df.drop(['team', 'opponent_team'], inplace = True, axis = 1)
    df = preprocess_data(df)
    min_date = df[(df['GW'] == GW) & (df['kickoff_time'] >= '2020-08-12')]['kickoff_time'].min() # The first date of the gameweek
    max_date = df[(df['GW'] == GW) & (df['kickoff_time'] > '2020-08-12')]['kickoff_time'].max() # The last date of the gameweek
    df_scl = df[df['kickoff_time'] <= max_date].copy() # Only includes up to gameweek 
    std_x, std_y = StandardScaler(), StandardScaler()
    scaled_cols = []
    if scale == True:
      for col in df.columns.drop(['total_points', 'kickoff_time']):
        if df[col].nunique() > 2:
          scaled_cols.append(col)
      df_scl[scaled_cols] = std_x.fit_transform(df_scl[scaled_cols])
      df_scl['total_points'] = std_y.fit_transform(df_scl['total_points'].to_numpy().reshape(-1, 1))
      df_train, df_test = df_scl[df_scl['kickoff_time'] < min_date], df_scl[df_scl['kickoff_time'] >= min_date] 
      X = df_train.drop(['total_points', 'kickoff_time', 'GW'], axis = 1)
      y = df_train['total_points']
      X_val = df_test.drop(['total_points', 'kickoff_time', 'GW'], axis = 1)
      y_val = df_test['total_points']
    else:
      df_train, df_test = df_scl[df_scl['kickoff_time'] < min_date], df_scl[df_scl['kickoff_time'] >= min_date] 
      X = df_train.drop(['total_points', 'kickoff_time', 'GW'], axis = 1)
      y = df_train['total_points']
      X_val = df_test.drop(['total_points', 'kickoff_time', 'GW'], axis = 1)
      y_val = df_test['total_points']
    return X, X_val, y, y_val, std_x, std_y

def write_grid_results(grid, file = 'results.txt'):
  f = open(f'/content/drive/MyDrive/Parameters/{file}', 'w')
  print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
  f.write("Best: %f using %s\n\n" % (grid.best_score_, grid.best_params_))
  means = grid.cv_results_['mean_test_score']
  stds = grid.cv_results_['std_test_score']
  params = grid.cv_results_['params'] 
  used = []
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))
      f.write("%f (%f) with: %r\n\n" % (mean, stdev, param))
      used.append(param)
  f.close()
  X_train, X_test, y_train, y_test, std_x, std_y = get_training_data(GW = 1, scale = True)


def get_stacked_mod(GW):
    X, X_val, y, y_val, std_x, std_y = get_training_data(GW, scale = True)
    print('Train', X.shape, y.shape, 'Test', X_val.shape, y_val.shape)
    models = get_models()
    meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
    # return meta_X, meta_y
    # print('Meta ', meta_X.shape, meta_y.shape)
    models = fit_base_models(X, y, models)
    meta_model = fit_meta_model(meta_X, meta_y)
    evaluate_models(X_val, y_val, models, std_y)
    yhat = super_learner_predictions(X_val, models, meta_model)
    # # regression_results(np.round(std_y.inverse_transform(y_val)), np.round(std_y.inverse_transform(yhat)), 'Stacked')
    # # plot_results(y_val, yhat, title = f'Testing Stacking for GW ')
    print('Super Learner: RMSE %.3f' % (sqrt(mean_squared_error(np.round(std_y.inverse_transform(y_val)), np.round(std_y.inverse_transform(yhat))))))
    # # print(pd.DataFrame({'True': np.round(std_y.inverse_transform(y_val)), 'Predicted': np.round(std_y.inverse_transform(yhat))}))
    # return np.round(std_y.inverse_transform(yhat)).flatten().tolist()

get_stacked_mod(1)

Train (10658, 58) (10658,) Test (220, 58) (220,)
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
Training LinearRegression
LinearRegression: RMSE 3.184
LinearRegression: RMSE 13.586
Super Learner: RMSE 3.182


In [22]:
models = get_models()
pd.DataFrame(x, columns = [model.__class__.__name__ for model in models])


,LinearRegression,LinearRegression
0,-0.893799,-0.893799
1,-0.608154,-0.608154
2,-0.795898,-0.795898
3,0.083740,0.083740
4,0.040283,0.040283
...,...,...
10653,-0.015778,-0.015778
10654,0.121376,0.121376
10655,0.203335,0.203335
10656,-0.026716,-0.026716
